In [34]:


from pipelines import pipeline
QGen = pipeline("question-generation")
MultiQGen= pipeline("multitask-qa-qg")
OnlyQ =  pipeline("e2e-qg")

In [2]:
import spacy
nlp = spacy.load('en_core_web_md')

In [32]:
data='''The two characteristic features seen in carbon, that is, tetravalency and catenation, put
together give rise to a large number of compounds. Many have the same non-carbon
atom or group of atoms attached to different carbon chains. These compounds were
initially extracted from natural substances and it was thought that these carbon
compounds or organic compounds could only be formed within a living system. That is,
it was postulated that a ‘vital force’ was necessary for their synthesis. Friedrich Wöhler
disproved this in 1828 by preparing urea from ammonium cyanate. But carbon
compounds, except for carbides, oxides of carbon, carbonate and hydrogencarbonate
salts continue to be studied under organic chemistry'''

In [37]:
data0='''

The Pythagorean theorem is one of the earliest theorems. This famous theorem is named after the Greek mathematician and
Philosopher, Pythagoras. Although the theorem was named after him but there is evidence that the Babylonians knew this relationship some 1000 years earlier. Plimpton 322, a Babylonian mathematical tablet dated back to 1900 B.C., contains a table of Pythagorean triples. The Chou-pei, an ancient Chinese text, also provides the evidence that the Chinese knew about the Pythagorean theorem many years before Pythagorean.

The Pythagorean Theorem is Pythagoras' most famous mathematical contribution. According to the legend, Pythagoras was so happy when he discovered the theorem that he offered a sacrifice of oxen. The later discovery that square root of 2 is irrational
and, therefore cannot be expressed as a ratio of two integers, greatly troubled Pythagoras and his followers.

The Pythagorean theorem states that: "The area of the square built on the hypotenuse of a right triangle is equal to the sum of the squares on the remaining two sides."
The key formula of the pythogras theorem is that (hypotenuse)^2 = (opposite)^2+(adjacent)^2.
This has many applications in real life problems
'''

In [4]:
def filter_length_answers(json_result):
    result = []
    for qa_pair in json_result:
        lenght_of_answer = len(qa_pair["answer"]) 
        if(lenght_of_answer> 50):
            continue
        result.append(qa_pair)
    return result

In [5]:
def ProcessQuestions(data):
    data.replace("\n", "")
    qs1 = QGen(data)
    result = json_result = filter_length_answers(qs1)
    return filter_length_answers(result)
   
  

In [11]:
def GetQuestionAnswer(json_result):
  l=[]
  for idx, qa_pair in enumerate(json_result):
      l.append([qa_pair['question'],qa_pair['answer']])
  return l



In [38]:
questions = ProcessQuestions(data0)
GetQuestionAnswer(questions)

[['What is one of the earliest theorems?', 'Pythagorean theorem'],
 ['Who is the name of the Pythagorean theorem?',
  'Greek mathematician and Philosopher, Pythagoras'],
 ['What year did the Babylonians know about the Pythagorean theorem?',
  '1000 years earlier'],
 ['What is the name of the Babylonian mathematical tablet that contains a table of Pythagorean triples?',
  'Plimpton 322'],
 ['What ancient Chinese text provides evidence that the Chinese knew about the Pythagorean theorem many years before Pythagorean',
  'Chou-pei'],
 ['What is Pythagoras most famous mathematical contribution?',
  'Pythagorean Theorem'],
 ['What did Pythagoras offer when he discovered the theorem?',
  'sacrifice of oxen'],
 ['What did Pythagoras discover when he offered a sacrifice of oxen?',
  'square root of 2 is irrational'],
 ['The Pythagorean theorem states that the area of the square built on what is equal to the sum of the squares on',
  'the hypotenuse of a right triangle'],
 ['What is the key for

In [10]:
data2='''
The Indian independence movement was a series of historic events with the ultimate aim of ending the British rule in India. The movement spanned from 1857 to 1947.[1] The first nationalistic revolutionary movement for Indian independence emerged from Bengal.[2] It later took root in the newly formed Indian National Congress with prominent moderate leaders seeking only their fundamental right to appear for Indian Civil Service examinations in British India, as well as more rights (economical in nature) for the people of the soil. The early part of the 20th century saw a more radical approach towards political self-rule proposed by leaders such as the Lal Bal Pal triumvirate, Aurobindo Ghosh and V. O. Chidambaram Pillai.[3]
The last stages of the self-rule struggle from the 1920s was characterized by Congress's adoption of Mahatma Gandhi's policy of non-violence and civil disobedience, and several other campaigns. Nationalists like Subhas Chandra Bose, Bhagat Singh, Bagha Jatin, Surya Sen preached armed revolution to achieve self-rule. Poets and writers such as Rabindranath Tagore, Subramania Bharati, Bankim Chandra Chattopadhyay and Kazi Nazrul Islam used literature, poetry, and speech as a tool for political awareness. Feminists like Sarojini Naidu, Pritilata Waddedar, Begum Rokeya promoted the emancipation of Indian women and their participation in national politics.[3] B. R. Ambedkar championed the cause of the disadvantaged sections of Indian society within the more significant self-rule movement.[4] The period of the World War II saw the peak of the campaigns by the Quit India Movement led by Congress and the Indian National Army movement led by Subhas Chandra Bose with the help of Japan
'''


In [11]:
data3 = '''
A condition that occurs when two processes are each waiting for the other to complete before proceeding. The result is that both processes hang. Deadlocks occur most commonly in multitasking and client/server environments. Ideally, the programs that are deadlocked, or the operating system, should resolve the deadlock, but this doesn't always happen. A deadlock is also called a deadly embrace.

The cause of deadlocks: Each process needing what another process has. This results from sharing resources such as memory, devices, links.

In an operating system, a deadlock is a situation which occurs when a process or thread enters a waiting state because a resource requested by it is being held by another waiting process, which in turn is waiting for another resource. If a process is unable to change its state indefinitely because the resources requested by it are being used by another waiting process, then the system is said to be in a deadlock.
'''

In [12]:
def short_question_generation(data):
   
    questions = ProcessQuestions(data)
    #print(questions)
    short_answers=[]
    for item in questions:
        ans = item['answer']
        ans = nlp(ans)
        for entity in ans.ents:
          q_dict={}
          if(entity.label_=='PERSON'):
            question = "Who was "+str(entity)+" ?"
            answer = MultiQGen({
                "question": question,
                "context": data
            })
            #answer = str(entity) +" was the person "+item['question']
            q_dict['question']=question
            q_dict['answer']=answer
            short_answers.append(q_dict)

          elif entity.label_=='DATE':
            question = "What event occured in "+str(entity)+" ?"
            answer = MultiQGen({
                "question": question,
                "context": data
            })
            #answer =  str(entity) +" was the time period "+item['question']
            q_dict['question']=question
            q_dict['answer']=answer
            short_answers.append(q_dict)

          elif entity.label_=='ORG' or entity.label_=='GPE':
            question = "What event occured in "+str(entity)+" ?"
            answer = MultiQGen({
                "question": question,
                "context": data
            })
            #answer =  str(entity) +" was the time period "+item['question']
            q_dict['question']=question
            q_dict['answer']=answer
            short_answers.append(q_dict)
          print(entity,entity.label_)
    return short_answers

In [13]:
short_answers = short_question_generation(data)
GetQuestionAnswer(short_answers)

Friedrich Wöhler PERSON
Question 1:
Who was Friedrich Wöhler ?

Press Enter to see the answer


Answer: preparing urea from ammonium cyanate

---------------------------


In [14]:
short_answers = short_question_generation(data3)
print(short_answers)

two CARDINAL
[]


In [15]:
OnlyQ(data)

['What two characteristic features are seen in carbon?',
 'What are the two characteristic characteristics of carbon called?',
 'How many compounds have the same non-carbon atom or group of atoms attached to different carbon chains?',
 'When did Friedrich Wöhler disprove this?']